<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/get_posts_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
!pip install -U pymorphy3
import pymorphy3
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя-')
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [ ]:
domains = ['rk_nationalmuseum','olonmus','museum_ptz','echo_association','domderevnivoknavolok']

df = pd.concat([pd.read_csv('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/' + domain +'.csv', usecols = ['text','date']) for domain in domains], ignore_index=True)
df = df[df['text'].notna() & (df['text'].apply(lambda x: isinstance(x, str)))]
df = df.reset_index()
df = df.drop(columns=['index'])


def process_text(text):
    check_hash = False
    processed_parts = []
    for w in nltk.word_tokenize(text):
      if len(w) == 1:
        continue
      if w == '#':
          check_hash = True
          continue
      if check_hash:
          check_hash = False
          continue
      w_tag = morph.parse(w.strip())[0].tag
      if   'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
         continue
      if set(w.lower()).issubset(alphabet):
        if w.isalpha() and w.lower():
          if w.isupper() and len(w) <= 3:
              processed_parts.append(w)
          else:
              res = morph.parse(w.lower())[0].normal_form
              if res not in stop_words:
                  processed_parts.append(res)
    result = ' '.join(processed_parts)
    return str(result)

df['tokens'] = df['text'].apply(lambda x: process_text(x))
df_tokens = pd.concat([df['tokens'], df['date']], axis=1, keys=['tokens', 'date'])

df_tokens.to_csv('tokens.csv', index=False)